In [1]:
import logging
import os
import re
from urllib.parse import urlparse, parse_qs
import tempfile


from bs4 import BeautifulSoup
import jinja2
import requests

from tessa_chef import make_request, get_parsed_html_from_url, make_fully_qualified_url
from tessa_chef import get_license, get_text
from tessa_chef import get_section_filename, create_predictable_zip, download_file
from tessa_chef import shutil

from le_utils.constants import content_kinds, file_formats, licenses

import pprint
pp = pprint.PrettyPrinter(indent=4, width=80)

In [4]:
test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108' # EN
# test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82912' # AR
# test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514' # FR

In [7]:

def download_module(module_url):
    print('Scrapring module @ url =', module_url)
    doc = get_parsed_html_from_url(module_url)
    destination = tempfile.mkdtemp()
    print('destination=', destination)

    # copy css/js/images from skel
    shutil.copytree('../chefdata/templates/module_skel/styles', os.path.join(destination,'styles'))

    source_id = parse_qs(urlparse(module_url).query)['id'][0]
    raw_title = doc.select_one("head title").text
    module_title = raw_title.replace('OLCreate:', '')\
            .replace('TESSA_ARABIC', '')\
            .replace('TESSA_Eng', '')\
            .replace('TESSA_Fr', '')\
            .strip()

    module_contents_dict = dict(
        kind='TessaModuleContentsDict',
        language='en', # TODO change me
        source_id=source_id,
        title=module_title,
        children=[],
    )

    is_first_section = True
    section_lis = doc.find('ul', class_='oucontent-tree-current-section').find_all('li', recursive=False)
    for section_li in section_lis:            
        section_link = section_li.find('a')
        section_href = section_link['href']
        # TODO: special case for first section --- since it doesn't save section in filename
        #       manually call download_page with filename section_1.html with contents of current page
        if is_first_section:
            section_filename = 'section_1.html'
            is_first_section = False
        else:
            section_filename = get_section_filename(section_href)
        subsections_ul = section_li.find('ul')
        accesshide_span = section_li.find('span', class_='accesshide')
        subsection_lis = subsections_ul.find_all('li')
        if accesshide_span:
            accesshide_span.extract()
        subsections_ul.extract()
        section_title = get_text(section_li)

        section_dict = dict(
            kind='TessaModuleSection',
            title=section_title,
            href=section_href,
            filename=section_filename,
            children=[],
        )
        module_contents_dict['children'].append(section_dict)

        for subsection_li in subsection_lis:
            subsection_link = subsection_li.find('a')
            subsection_href = subsection_link['href']
            subsection_filename = get_section_filename(subsection_href)
            subaccesshide_span = subsection_li.find('span', class_='accesshide')
            if subaccesshide_span:
                subaccesshide_span.extract()
            subsection_title = get_text(subsection_li)
            subsection_dict = dict(
                kind='TessaModuleSubsection',
                title=subsection_title,
                href=subsection_href,
                filename=subsection_filename,
            )
            section_dict['children'].append(subsection_dict)

    module_index_tmpl = jinja2.Template(open('../chefdata/templates/module_index.html').read())
    index_contents = module_index_tmpl.render(module=module_contents_dict)
    with open(os.path.join(destination, "index.html"), "w") as f:
        f.write(index_contents)

    # download the html content from each section/subsection
    for section in module_contents_dict['children']:
        download_page(section['href'], destination, section['filename'], module_contents_dict, section)
        for subsection in section['children']:
            download_page(subsection['href'], destination, subsection['filename'], module_contents_dict, subsection)

    return module_contents_dict

    zip_path = create_predictable_zip(destination)
    print('zip_path =', zip_path)
    module_node = dict(
        kind='HTML5AppNode',
        source_id=source_id,
        language=module_contents_dict['language'],
        title=title,
        license=get_license(licenses.CC_BY_NC_SA, copyright_holder='TESSA'),
        description='fake descri',
        files=[{'file_type':'HTMLZipFile', 'path':zip_path}],
    )
    return module_node



def download_page(page_url, destination, filename, module_dict, page_info):
    print('Scrapring section/subsectino...', filename)
    doc = get_parsed_html_from_url(page_url)
    source_id = parse_qs(urlparse(page_url).query)['id'][0] + '/' + filename   # or should I use &section=1.6 ?

    
    # We're only interested in the main content inside the section#region-main
    main_region = dict(
        args=['section'],
        kwargs={'id': 'region-main'},
    )
    section = doc.find(*main_region['args'], **main_region['kwargs'])

    
    def download_assets(selector, attr, middleware=None):
        """
        Download all assets refered to in `attr` for dom elements that match
        `selector` within of section (taken from download_page's scope).
        """
        print('donwloading all', selector, 'assets')
        nodes = section.select(selector)

        for i, node in enumerate(nodes):
            url = make_fully_qualified_url(node[attr])
            filename = "%s_%s" % (i, os.path.basename(url))
            node[attr] = filename
            download_file(url, destination, request_fn=make_request, filename=filename, middleware_callbacks=middleware)

    def js_middleware(content, url, **kwargs):
        return content

    # Download all static assets
    download_assets("img[src]", "src")     # Images
    download_assets("link[href]", "href")  # CSS
    download_assets("script[src]", "src", middleware=js_middleware) # JS

    raw_title = doc.select_one("head title").text
    section_title = raw_title.replace('OLCreate:', '')\
            .replace('TESSA_ARABIC', '')\
            .replace('TESSA_Eng', '')\
            .replace('TESSA_Fr', '')\
            .strip()

    section_dict = dict(
        title=section_title,
        language=module_dict['language'],
        main_content=str(section),
    )

    section_index_tmpl = jinja2.Template(open('../chefdata/templates/section_index.html').read())
    index_contents = section_index_tmpl.render(
        module = module_dict,
        section = section_dict,
    )
    with open(os.path.join(destination, filename), "w") as f:
        f.write(index_contents)


In [8]:
toc = download_module(test_module)



Scrapring module @ url = http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108
destination= /var/folders/k3/r74jr38d56v717n39fd073f80000gn/T/tmpx7agda4u
Scrapring section/subsectino... section_1.html
donwloading all img[src] assets
donwloading all link[href] assets
donwloading all script[src] assets
Scrapring section/subsectino... section_1_1.html
donwloading all img[src] assets
donwloading all link[href] assets
donwloading all script[src] assets
Scrapring section/subsectino... section_1_2.html
donwloading all img[src] assets
donwloading all link[href] assets
donwloading all script[src] assets
Scrapring section/subsectino... section_1_3.html
donwloading all img[src] assets
donwloading all link[href] assets
donwloading all script[src] assets
Scrapring section/subsectino... section_1_4.html
donwloading all img[src] assets
donwloading all link[href] assets
donwloading all script[src] assets
Scrapring section/subsectino... section_1_5.html
donwloading all img[src] assets
donw

In [9]:
for section in toc['children']:
    print('  - section:', section['title'], section['filename'])
    for subsection in section['children']:
        print('    - subsection:', subsection['title'], subsection['filename'])



  - section: Section 1: Ways to explore who pupils are section_1.html
    - subsection: 1. Organising pupils in groups and pairs section_1_1.html
    - subsection: 2. Finding out what pupils think and feel section_1_2.html
    - subsection: 3. Respecting differences section_1_3.html
    - subsection: Resource 1: Similarities and differences section_1_4.html
    - subsection: Resource 2: Asking questions about feelings section_1_5.html
    - subsection: Resource 3: How to conduct a class survey section_1_6.html
    - subsection: Resource 4: Observing your pupils section_1_7.html
    - subsection: Acknowledgements section_1_8.html
  - section: Section 2: Planning physical growth and development sessions section_2.html
    - subsection: 1. Focus on planning section_2_1.html
    - subsection: 2. Using games in your teaching section_2_2.html
    - subsection: 3. Teaching about healthy living section_2_3.html
    - subsection: Resource 1: Introduction to physical development section_2_4.html

In [5]:

def passa():


